# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [62]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats


In [ ]:
df = pd.read_csv('homepage_actions.csv')
df.columns

In [10]:
df.head(5)


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [19]:
clickers = set(df[df.action=='click']['id'].unique())
viewers = set(df[df.action=='view']['id'].unique())
l = len(clickers-viewers)
l

0

no clickers who didnt also view

In [16]:
exp_id = set(df[df.group=='click']['id'].unique())
cont_id = set(df[df.group=='click']['id'].unique())
any(i in exp_id for i in cont_id)

0


False

no overlap

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [53]:
#Your code here
df['count'] = 1
df_g = df.groupby(['group', 'action']).sum()
df_g

id  count
group      action                   
control    click    525683507    932
           view    1874171236   3332
experiment click    528944090    928
           view    1694962719   2996

In [54]:
df_g.index

MultiIndex([(   'control', 'click'),
            (   'control',  'view'),
            ('experiment', 'click'),
            ('experiment',  'view')],
           names=['group', 'action'])

In [57]:
c_click = df_g.loc[('control', 'click'), 'count']
c_view = df_g.loc[('control', 'view'), 'count']
e_click = df_g.loc[('experiment', 'click'), 'count']
e_view = df_g.loc[('experiment', 'view'), 'count']

c_rate = c_click/(c_view)
e_rate = e_click/e_view
print(c_rate, e_rate)

0.2797118847539016 0.3097463284379172


In [64]:
c_array = np.hstack((np.ones(c_click), np.zeros((c_view-c_click))))
e_array = np.hstack((np.ones(e_click), np.zeros((e_view-e_click))))



0.2797118847539016

In [67]:
p = stats.ttest_ind(c_array, e_array)[1]/2
p

0.0044125494574791466

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [58]:
#Your code here
expected = e_view*c_rate
expected

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [71]:
#Your code here
sd_c = (e_view * c_rate * (1 - c_rate))**0.5
sd_c

24.568547907005815

In [73]:
z_value = (e_click-expected)/sd_c
z_value

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [74]:
#Your code here
p_value =  stats.norm.sf(z_value)
p_value

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Although there is a differnce between the two p values, both are very small and give us significant results to reject the hypothesis that the control and experiment had the same mean. We can say the change has had a positive effect**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.